# header file

In [29]:
import numpy as np
import pandas as pd
print('done')

done


In [30]:
np.set_printoptions(suppress=True) #prevent numpy exponential 

In [31]:
path = "/media/mint/New Volume/4_semester_IIIT_D/capstone/hmog_dataset/171538/171538_session_"#13/TouchEvent.csv'

In [32]:
# screen grid (splitting the user screen into y_cross * x_cross)
y_cross = 8
x_cross = 6

# getting the screen size

In [33]:
# to get the minimum and maximum dimension of user screen size 

def screen_size():
    xx_min = 100000
    xx_max = -1
    yy_min = 100000
    yy_max = -1
    for i in range(1,25):
        data = pd.read_csv(path+str(i)+"/TouchEvent.csv", header=None)
        # data.head()
        x_min = np.min(data.iloc[:,6])# 6 and 7 th are the x and y axis of csv file
        x_max = np.max(data.iloc[:,6])
        y_min = np.min(data.iloc[:,7])
        y_max = np.max(data.iloc[:,7])
        if(x_min < xx_min):
            xx_min = x_min
        if(y_min < yy_min):
            yy_min = y_min
        if(x_max > xx_max):
            xx_max = x_max
        if(y_max > yy_max):
            yy_max = y_max
    # print(xx_min)
    # print(xx_max)
    # print(yy_min)
    # print(yy_max)

    x_len = xx_max - xx_min
    # print(x_len)
    fact = x_len / x_cross 
    # print(fact)
    x_axis = np.linspace(fact, x_len-fact, x_cross-1)
    print(x_axis)

    y_len = yy_max - yy_min
    # print(y_len)
    fact = y_len / y_cross 
    # print(fact)
    y_axis = np.linspace(fact, y_len-fact, y_cross-1)
    print(y_axis)
    return x_axis, y_axis

In [34]:
x_axis, y_axis = screen_size()

[ 174.56558967  349.13117933  523.696769    698.26235867  872.82794833]
[  227.25   454.5    681.75   909.    1136.25  1363.5   1590.75]


# POLICY iteration: parameter initialisation

In [35]:

NO_OF_STATES = 48#24#36#80#48#
NO_OF_ACTIONS = 2# 0 means remain in same state, 1 otherwise  
TRANSITION_PROBABILITY= np.zeros((NO_OF_STATES, NO_OF_ACTIONS, NO_OF_STATES))  # transition probability
TRANSITION_PROBABILITY= TRANSITION_PROBABILITY+ 0.00001
print(TRANSITION_PROBABILITY.shape)
REWARD = np.zeros((NO_OF_STATES, NO_OF_ACTIONS, NO_OF_STATES))  # rewards
REWARD =REWARD + 0.00001 # - 10.1
print(REWARD.shape)

POLICY = [0 for s in range(NO_OF_STATES)]
V = np.zeros(NO_OF_STATES)

GAMMA = 0.7

(48, 2, 48)
(48, 2, 48)


# POLICY iteration

In [36]:
def POLICY_iteration():
    flag = True # whether value changed or not

    while flag:
        flag = False

        for s in range(NO_OF_STATES):
            temp = 0
            for s1 in range(NO_OF_STATES):
                temp += (REWARD[s,POLICY[s],s1] + GAMMA*V[s1]) * TRANSITION_PROBABILITY[s,POLICY[s],s1]   
            V[s] = temp
                
        for s in range(NO_OF_STATES):
            cur = V[s]

            for a in range(NO_OF_ACTIONS):
                temp = 0
                for s1 in range(NO_OF_STATES):
                    temp += (REWARD[s, a, s1] + GAMMA * V[s1]) * TRANSITION_PROBABILITY[s, a, s1] 
                
                if temp > cur:
                    flag = True
                    POLICY[s] = a
                    cur = temp


    print "POLICY:  "
    policy_temp = np.zeros((y_cross,x_cross))
    v_temp = np.zeros((y_cross,x_cross))
    for i in range(y_cross):
        for j in range(x_cross):
            #print(i,j), 
#             print(i*x_cross+j),
            policy_temp[i,j] = POLICY[i*x_cross+j]
            v_temp[i,j] = V[i*x_cross+j]

    print policy_temp
    print v_temp

# Initially data is in milisecond
    
    so milisecond must be combined to form some meaningful things
    
# So, Milisecond data is combined into .1 seconds

In [37]:
def user_data_generation(start_user_id, end_user_id):
    user_movement_data = []

    count = 0
    for k in range(start_user_id, end_user_id+1):
        data = pd.read_csv(path+str(k)+"/TouchEvent.csv", header=None)
        #count = 0
        i = 0
        #for i in range(len(data)-1):
        while i < len(data): 
            #count += 1# total number of rows
            x_cor = []
            y_cor = []
            total_pressure = 0

            time1 = data.iloc[i,1]

            j = i
            while j < len(data):

                time2 = data.iloc[j,1]

                if time2-time1<=100: # .1 seconds session is combined
                    x = data.iloc[j,6]
                    y = data.iloc[j,7]
                    pressure = data.iloc[j,8]

                    x_cor.append(x)
                    y_cor.append(y)
                    total_pressure += pressure    

                    j+= 1

                else:
                    break

            x = np.mean(x_cor)
            y = np.mean(y_cor)

            user_movement_data.append([x,y,total_pressure])

            count = count +1
            i = j

    print(count)

    user_movement_data = pd.DataFrame(user_movement_data)

    return user_movement_data
                 

# Initial 20 session for training

# Last 4 session for testing

In [38]:
user_movement_data = user_data_generation(1,20)
print(user_movement_data.head())
user_movement_data_test = user_data_generation(21,24)
print(user_movement_data_test.head())

19357
            0            1    2
0  756.632357   960.230617  6.0
1  520.156514  1046.569620  5.0
2  341.169537  1158.260014  7.0
3  296.000000  1210.000000  1.0
4  515.287265  1113.390633  6.0
1624
            0            1    2
0  199.333333   954.666667  9.0
1  138.857143  1158.000000  7.0
2  202.090297   175.670067  6.0
3  203.028040   169.000000  3.0
4   71.846525   137.793798  6.0


# POLICY iteration is applied on data after 1 second time interval

In [39]:
alpha = 0.5
# for j in range(1,2):
#data = pd.read_csv(path+str(j)+"/TouchEvent.csv", header=None)
count = 0
for i in range(len(user_movement_data)-1):
    x = np.digitize(x = [user_movement_data.iloc[i,0]], bins = x_axis)[0]
    y = np.digitize(x = [user_movement_data.iloc[i,1]], bins = y_axis)[0]

    x_next = np.digitize(x = [user_movement_data.iloc[i+1,0]], bins = x_axis)[0]
    y_next = np.digitize(x = [user_movement_data.iloc[i+1,1]], bins = y_axis)[0]

    pressure = user_movement_data.iloc[i,2]
    #contact_size = data.iloc[i,9]

    state1 = y*x_cross+x
    state2 = y_next * x_cross+ x_next

    if state1 == state2:
        action = 0
    else:
        action = 1

    TRANSITION_PROBABILITY[state1][action][state2] += 1#0.001#1
    REWARD[state1][action][state2] += 100 + alpha * pressure#0.001 * pressure#(pressure * contact_size)

    count = count + 1
    if count % 600 == 0: #  # 50 times for 5 seconds, 600 times for 1 minutes(60 seconds)

        for k in range(NO_OF_STATES):
            TRANSITION_PROBABILITY[k][0] = TRANSITION_PROBABILITY[k][0]/ np.sum(TRANSITION_PROBABILITY[k][0])
            TRANSITION_PROBABILITY[k][1] = TRANSITION_PROBABILITY[k][1]/ np.sum(TRANSITION_PROBABILITY[k][1])

        POLICY_iteration()


# POLICY_iteration()
for k in range(NO_OF_STATES):
    TRANSITION_PROBABILITY[k][0] = TRANSITION_PROBABILITY[k][0]/ np.sum(TRANSITION_PROBABILITY[k][0])
    TRANSITION_PROBABILITY[k][1] = TRANSITION_PROBABILITY[k][1]/ np.sum(TRANSITION_PROBABILITY[k][1])

# POLICY_iteration()

print(count)
print('done')

POLICY:  
[[ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.]
 [ 1.  1.  1.  1.  1.  1.]
 [ 1.  0.  0.  0.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.]
 [ 0.  1.  1.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]]
[[  6634.43594514   6634.43594514   6634.43594514   6634.43594514
    6634.43594514   6634.43594514]
 [  6634.43594514   6634.43594514   6634.43594514   6634.43594514
    6634.43594514   6634.43594514]
 [  6634.43594514   6634.43594514  14580.42261916   6634.43594514
    6634.43594514   6634.43594514]
 [ 12340.65036828  10778.87948177  10457.86552825  12698.21711503
   13271.14804253   7972.35448297]
 [ 11318.18922692  20662.96271651  20689.62238251  22614.60406272
   12292.95230524  12699.06461129]
 [  7971.85471785  11191.94659199  11244.99379838  10474.1688268
   12525.28262536   8069.8989126 ]
 [  6634.43594514  14580.42261916   8074.15115222  15927.76307688
    6634.43594514   6634.43594514]
 [  6634.43594514   6634.43594514   6634.43594514   6634.43594514
 

# testing phase

In [40]:
# error = 0.0
correct_count = 0
acc = []


time_count = 0
for i in range(len(user_movement_data_test)-1):
    x = np.digitize(x = [user_movement_data_test.iloc[i,0]], bins = x_axis)[0]
    y = np.digitize(x = [user_movement_data_test.iloc[i,1]], bins = y_axis)[0]

    x_next = np.digitize(x = [user_movement_data_test.iloc[i+1,0]], bins = x_axis)[0]
    y_next = np.digitize(x = [user_movement_data_test.iloc[i+1,1]], bins = y_axis)[0]

    pressure = user_movement_data_test.iloc[i,2]
    #contact_size = data.iloc[i,9]

    state1 = y*x_cross+x
    state2 = y_next * x_cross+ x_next

    if state1 == state2:
        action = 0
    else:
        action = 1
    
    # checking for whether user have taken right action or not
    if POLICY[state1] == action:
        correct_count += 1
        
    #comment
    TRANSITION_PROBABILITY[state1][action][state2] += 1#0.001#1
    REWARD[state1][action][state2] += 100 + 0.5 * pressure#0.001 * pressure#(pressure * contact_size)

    time_count = time_count + 1
    if time_count % 300 == 0: #  # 50 times for 5 seconds, 600 times for 1 minutes(60 seconds)

        acc_in = (100*(correct_count))/300.0
        acc.append(acc_in)
        
        correct_count = 0

        # comment
        for k in range(NO_OF_STATES):
            TRANSITION_PROBABILITY[k][0] = TRANSITION_PROBABILITY[k][0]/ np.sum(TRANSITION_PROBABILITY[k][0])
            TRANSITION_PROBABILITY[k][1] = TRANSITION_PROBABILITY[k][1]/ np.sum(TRANSITION_PROBABILITY[k][1])

        POLICY_iteration()


# comment
for k in range(NO_OF_STATES):
    TRANSITION_PROBABILITY[k][0] = TRANSITION_PROBABILITY[k][0]/ np.sum(TRANSITION_PROBABILITY[k][0])
    TRANSITION_PROBABILITY[k][1] = TRANSITION_PROBABILITY[k][1]/ np.sum(TRANSITION_PROBABILITY[k][1])

POLICY_iteration()


print(time_count)


acc_in = (100.0 *(correct_count))/ (time_count%300)
acc.append(acc_in)


POLICY:  
[[ 1.  1.  0.  0.  0.  1.]
 [ 1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.]
 [ 1.  1.  0.  0.  1.  1.]
 [ 1.  0.  0.  0.  0.  1.]
 [ 1.  0.  1.  0.  0.  1.]
 [ 1.  1.  1.  1.  1.  1.]
 [ 0.  1.  1.  1.  0.  0.]]
[[ 202338.0294409   138827.58870884  146906.28344172  146906.28344172
   146906.28344172  149257.91601114]
 [ 234737.4624719   164499.0487736   136478.94359764  123266.77176918
   138340.29182874  195661.32813995]
 [ 220294.55142186  180216.99536241  206106.16595918  189999.25943135
   182241.77634008  203919.61472932]
 [ 226458.29441905  220446.5616071   314420.00003333  378405.00003333
   218948.63117191  180357.08078615]
 [ 261656.44798879  321548.33336667  375745.00003333  496458.33336667
   255198.33336667  165743.23500198]
 [ 166374.85348586  200921.6667      183444.66185087  273361.6667
   435361.6667      175676.21687707]
 [ 131545.53398112  157500.71774318  207128.59369456  193402.63549623
   309883.82808666  184378.4787158 ]
 [ 146906.28344172  186046.8

In [41]:
# with modification
print(acc)
print(np.sum(acc)/len(acc))

[43.666666666666664, 53.666666666666664, 51.0, 47.666666666666664, 49.0, 35.77235772357724]
46.7953929539


In [42]:
# [44.666666666666664, 55.333333333333336, 51.333333333333336, 35.0, 48.0, 34.959349593495936]
# 44.8821138211